In [6]:
import cv2
import opsfaz as faz
import numpy as np
import drawfaz as draw

for precision in [0.5]:
    input_image1 = "/home/charlie/projects/entropy/raw/OCTA_Net_1000/treatment/inactive/499_2900821-7_20161013_OCT_00001331.bmp_superficial.jpg"
    input_image2 = "/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/104_1296950-5_20161031_OCT_00001000.bmp_superficial.jpg"

    # read image
    image1 = cv2.imread(input_image1,0)
    size1 = image1.shape
    image2 = cv2.imread(input_image2,0)
    size2 = image2.shape

    # configure parameters
    mm = 3
    deep = 0
    

    # call the function
    faz_image1, area, cnt = faz.detectFAZ(image1, mm, deep, precision) 
    faz_image2, area, cnt = faz.detectFAZ(image2, mm, deep, precision) 
    #	Outputs:
    #	- faz is a binary image with the region of the FAZ as mask
    #	- area is the area of the FAZ
    #	- cnt is the contour in opencv that represents the contour of the FAZ



    # we obtain the faz mask
    mask = cv2.drawContours(image.copy(), cnt, -1, (0,0,0), -1)
    #cv2.imshow("Mask extracted",faz)
    #cv2.imshow("Faz extracted",mask)
    faz2551 = faz_image1*255
    faz2552 = faz_image2*255
    cv2.imwrite("/home/charlie/projects/entropy/test/faz1"+ str(precision) +".jpg", faz2551)
    cv2.imwrite("/home/charlie/projects/entropy/test/faz2" + str(precision) + ".jpg", faz2552)
    #cv2.waitKey(1000)
    #cv2.destroyAllWindows()

    #print(type(cnt))
    print ("Area:	"+ str(area)+" square mm")

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/shapedescr.cpp:315: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'contourArea'


In [3]:
def fill_socket(image):
    image_ = image.copy()
    h, w = image.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    isbreak = False
    for i in range(image_.shape[0]):
        for j in range(image_.shape[1]):
            if(image_[i][j]==0):
                seedPoint=(i,j)
                isbreak = True
                break
        if(isbreak):
            break
    cv2.floodFill(image_, mask,seedPoint, 255)
    im_floodfill_inv = cv2.bitwise_not(image_)
    image = image | im_floodfill_inv
    return image

In [ ]:
def imgmerge(baseimg, coverimg, color = np.array([71, 130, 255, 60])):
    coverimg_draw = np.zeros(coverimg.shape+(4,)).astype('uint8')
    for i in range(4):
        coverimg_draw[:, :, i][coverimg > 0] = color[i]
    coverimg_draw = Image.fromarray(coverimg_draw)
    final = Image.alpha_composite(baseimg, baseimg)
    compose = Image.alpha_composite(final, coverimg_draw)
    return compose

In [10]:
# 這是一個check有沒有找到cnt的程式
# -*- encoding: utf-8 -*-
import cv2
import drawfaz as draw
import numpy as np
from skimage.morphology import skeletonize
from util import *

input_image1 = "/home/charlie/projects/entropy/raw/OCTA_Net_1000/treatment/inactive/499_2900821-7_20161013_OCT_00001331.bmp_superficial.jpg"
imOCTorig = cv2.imread(input_image1,0)

mm = 3
prof = 0
size = imOCTorig.shape
imOCT = np.zeros((size[0],size[1]),np.float64)
if (len(size) > 2):
	imOCT[:,:] = imOCTorig[:,:,0] / float(imOCTorig.max())
else:
	imOCT[:,:] = imOCTorig / float(imOCTorig.max())

# tophat
ss = ((size[0]/80)*2-mm+3)
ss = ss*2-1
imOCT = morph('tophat',imOCT,ss)

# canny edge detector
edges, imClosed = edges_extraction (imOCT,prof,mm,precision)
im = imClosed.copy()
# mask of the FAZ
	
fazMask = find_mask(im,prof,mm)

# select the highest candidate
image = fazMask.copy()
image1 = cv2.convertScaleAbs(image) 
contours, hierachy = cv2.findContours(image1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cogidos = []
cnt,cogidos = higest_contour(contours, cogidos)
print(cnt)
m = cv2.contourArea(cnt)
fazAreainMM = m*(mm*mm)/(size[0]*size[1])

im = np.zeros((size[0],size[1]), np.uint8)
cv2.drawContours(im, [cnt], 0, (255,255,255), -1)
im = im[:]/255
	
# make region growing
	
reg = region_growing(imOCT, im*1.0, fazAreainMM, prof, 4, precision)

reg = morph ('open', reg, 3)
reg = morph ('closed', reg, 3)
image1 = cv2.convertScaleAbs(reg) 
contours, hierachy = cv2.findContours(image1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cogidos = []
cnt,cogidos = higest_contour (contours, cogidos)

m = cv2.contourArea(cnt)
fazAreainMM = m*(mm*mm)/(size[0]*size[1])

octLabel = np.zeros((size[0],size[1],3))
octLabel[:,:,0] = imOCT*(1-(reg*0.5)) + reg*0.5
octLabel[:,:,1] = imOCT*(1-(reg*0.3)) + reg*0.35
octLabel[:,:,2] = imOCT*(1-reg)
im = reg[:]
faz =  im[:]

fazAreainUM = fazAreainMM * 1000
fazAreainUM = float("{0:.2f}".format(fazAreainUM))
M = cv2.moments(cnt)
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])

imOCT_area = draw.draw_in_image(imOCTorig, fazAreainUM, cnt, (50,50))
im_cnt = draw.draw_in_image(imOCTorig, 0, cnt, None)    

None


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/shapedescr.cpp:315: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'contourArea'
